In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df_userXproduct = pd.read_csv('data/userXproduct.csv',index_col=['user_id','product_id'],#nrows=10000,
                             dtype={'user_id':np.uint32,
                                    'product_id':np.uint32,
                                    'mean_atco':np.float32,
                                    'mean_order_how':np.float32,
                                    'number_reord':np.uint8,
                                    'dblo_mean':np.float32,
                                    'dslo':np.uint8,
                                    'mean_order_hod':np.float32,
                                    'mean_order_dow':np.float32,
                                    'reord_per_aisle':np.uint32,
                                    'reord_per_depart':np.uint32,
                                    'train_order_number':np.uint8,
                                    'train_dspo':np.float16,
                                    'reord_per_user':np.uint16,
                                    'reord_per_dow':np.uint32,
                                    'reord_per_hod':np.uint32,
                                    'reord_per_how':np.uint32,})

In [3]:
df_target  = pd.read_csv('data/target_train.csv',index_col=['user_id','product_id'])
df_userXproduct = df_userXproduct.join(df_target)
df_userXproduct.fillna(False,inplace=True)

In [4]:
df_userXproduct['reordered_ratio'] = df_userXproduct['number_reord'] /(df_userXproduct['train_order_number']-1.)

In [5]:
df_userXproduct.columns.values

array(['mean_atco', 'mean_order_hod', 'mean_order_how', 'number_reord',
       'dblo_mean', 'dslo', 'mean_order_dow', 'reord_per_aisle',
       'reord_per_depart', 'train_order_number', 'train_dspo',
       'reord_per_user', 'reord_per_dow', 'reord_per_hod', 'reord_per_how',
       'reordered', 'reordered_ratio'], dtype=object)

In [6]:
X = df_userXproduct.reset_index()[['user_id',
                                   'product_id',
                                   'mean_atco', 
                                   'mean_order_how', 
                                   'number_reord', 
                                   'dblo_mean', 
                                   'dslo',
                                   'mean_order_hod',
                                   'mean_order_dow', 
                                   'reord_per_aisle', 
                                   'reord_per_depart',
                                   'train_order_number', 
                                   'train_dspo', 
                                   'reord_per_user',
                                   'reord_per_dow', 
                                   'reord_per_hod', 
                                   'reord_per_how', 
                                   'reordered_ratio'
                                    ]].values

y = df_userXproduct.reset_index()[['user_id','product_id','reordered']].values

print(X.shape)
print(y.shape)

(4833292, 18)
(4833292, 3)


# Neural net

In [7]:
from keras.models import Sequential
from keras.layers import Dense

# fix random seed for reproducibility
np.random.seed(7)
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.utils.np_utils import to_categorical

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import scale

Using TensorFlow backend.


In [8]:
X[:,2:] = scale(X[:,2:],axis=0)

# Prediction

In [9]:
#from keras.models import load_model
#model = load_model('model_file2.h5')

#y_pred = np.append(y,model.predict(X[:,2:]),axis=1)

In [20]:
import xgboost as xgb
# load model from file
loaded_model = xgb.load_model("model_xgb.xgb")
#y_pred = np.append(y,loaded_model.predict_proba(X[:,2:]),axis=1)

AttributeError: 'Booster' object has no attribute 'predict_proba'

In [ ]:
dtest  = xgb.DMatrix(X[:,2:])
preds = loaded_model.predict(dtest)

In [23]:
df_resultats = pd.DataFrame(y,columns=['user_id','product_id','test'])
df_resultats['pred_false'] = 0. 
df_resultats['pred_true'] = preds

In [ ]:
df_resultats = pd.DataFrame(y_pred,columns=['user_id','product_id','test','pred_false','pred_true'])
                        

In [24]:
df_resultats = df_resultats.set_index(['user_id','product_id'])
df_resultats['test'] = df_resultats['test'].astype(np.bool)
#df_resultats['pred_false'] = df_resultats['pred_false'].astype(np.float32)
df_resultats['pred_true'] = df_resultats['pred_true'].astype(np.float32)
del df_resultats['pred_false']
#df_resultats.head()

In [25]:
print ('Reading the orders dataset...')
df_orders = pd.read_csv('data/orders.csv',dtype={'order_id':np.uint32,
                                                 'user_id':np.uint32,
                                                 'order_number':np.uint8,
                                                 'order_dow':np.uint8,
                                                 'order_hour_of_day':np.uint8,
                                                 'days_since_prior_order':np.float16})

df_orders_test = df_orders.loc[df_orders['eval_set']=='test']

Reading the orders dataset...


In [26]:
def output_pred(df_resultats,decision_threshold):
    df_res = df_resultats.loc[df_resultats['pred_true']>decision_threshold].reset_index()
    df_res = df_res.groupby(['user_id'])['product_id'].apply(lambda x : ' '.join(x.astype(str)))

    df_a_soumettre = df_orders_test.join(df_res,on='user_id')[['order_id','product_id']].sort_values('order_id')
    df_a_soumettre = df_a_soumettre.fillna('None')

    df_a_soumettre.to_csv('soumission.csv',sep=',',
                                 index=False,
                                 header=['order_id','products'],
                                 quoting=False)
    return df_a_soumettre

In [27]:
df = output_pred(df_resultats,0.68)
df.shape

(75000, 2)